# Install Required Libraries

Install all necessary libraries for LangChain, OpenAI, ChromaDB, PDF parsing, and experimental tools.

In [1]:
!pip install langchain langchain-core langchain-community langchain-openai openai chromadb pypdf langchain-experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 90.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.2

# Import Libraries

Import all core libraries for OpenAI integration, LangChain LLMs, prompts, message types, output parsers, document loaders, text splitters, vector stores, retrieval, and Pydantic for structured outputs.

In [58]:
# OpenAI SDK
from openai import OpenAI

# LangChain LLM + Embedding integration
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# LangChain Core Building Blocks
from langchain_core.prompts import (
    PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, FewShotPromptTemplate
)
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.documents import Document
from langchain_core.example_selectors import LengthBasedExampleSelector

# Text splitting
from langchain.text_splitter import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Vector store
from langchain.vectorstores import Chroma

# Retrieval
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

# Loaders
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader

# Pydantic for structured parsing
from pydantic import BaseModel, Field


# Set OpenAI API Key and Define a Simple Generate Function

Set your OpenAI API key, initialize the OpenAI client, and define a helper function to generate completions using the OpenAI chat API.

In [ ]:
# Set your OpenAI API key
open_api_key = "Your_API_Key"
max_tokens=256

# Initialize the OpenAI client with your API key
client = OpenAI(api_key= open_api_key)


# Define a generate() method to mimic model.generate()
def generate(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.5,
        max_tokens=max_tokens
    )
    
    return {
        'results': [{
            'generated_text': response.choices[0].message.content
        }]
    }

# Generate a Simple Completion

Use the `generate` function to get a completion from the OpenAI API for a sample prompt.

In [60]:
msg = generate("In today's sales meeting, we ")
print(msg['results'][0]['generated_text'])

I'm here to help! It seems like your message got cut off. How can I assist you with your sales meeting today?


# Create and Use a LangChain LLM Wrapper

Wrap the OpenAI chat model with LangChain's `ChatOpenAI` class and invoke it directly for a simple question.

In [150]:

# Create an OpenAI LLM wrapper
openai_llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.5,
    openai_api_key= open_api_key,
    max_tokens=max_tokens
)

# Call it like before
response = openai_llm.invoke("Who is man's best friend?")
print(response.content)

A dog is often considered man's best friend.


# Structured Chat Messages Example

Demonstrate how to use structured chat messages with system and human roles for more controlled conversations.

In [62]:

# Structured chat messages (same as Watsonx)
msg = openai_llm.invoke([
    SystemMessage(content="You are a helpful AI bot that assists a user in choosing the perfect book to read in one short sentence"),
    HumanMessage(content="I enjoy mystery novels, what should I read?")
])

# Print the reply
print(msg.content)

You should read "The Girl with the Dragon Tattoo" by Stieg Larsson for a gripping and complex mystery story.


# Multi-Turn Structured Chat Example

Show a multi-turn conversation with system, human, and AI messages to simulate a dialogue.

In [63]:
# Structured chat messages (same as Watsonx)
msg = openai_llm.invoke([
        SystemMessage(content="You are a supportive AI bot that suggests fitness activities to a user in one short sentence"),
        HumanMessage(content="I like high-intensity workouts, what should I do?"),
        AIMessage(content="You should try a CrossFit class"),
        HumanMessage(content="How often should I attend?")
])

# Print the reply
print(msg.content)

It's recommended to attend CrossFit classes 3-5 times per week for optimal results.


# Simple Human Message Example

Send a single human message to the LLM and print the response.

In [64]:
# Structured chat messages (same as Watsonx)
msg = openai_llm.invoke([
        HumanMessage(content="What month follows June?")
])

# Print the reply
print(msg.content)

July


# Prompt Template Example

Create a prompt template with placeholders and render it with input values.

In [65]:
prompt = PromptTemplate.from_template("Tell me one {adjective} joke about {topic}")
input_ = {"adjective": "funny", "topic": "cats"}  # create a dictionary to store the corresponding input to placeholders in prompt template

prompt.invoke(input_)

StringPromptValue(text='Tell me one funny joke about cats')

# Chat Prompt Template Example

Use a chat prompt template to structure a conversation and render it with input values.

In [66]:

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

input_ = {"topic": "cats"}

prompt.invoke(input_)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})])

# Chat Prompt with Message Placeholders

Demonstrate how to use message placeholders in chat prompts for dynamic message insertion.

In [67]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder(variable_name="msgs")
])

input_ = {"msgs": [HumanMessage(content="What is the day after Tuesday?")]}

prompt.invoke(input_)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the day after Tuesday?', additional_kwargs={}, response_metadata={})])

# Compose and Run a LangChain Chain

Combine a prompt and LLM into a chain and run it with input data.

In [68]:
# Compose the chain
chain = prompt | openai_llm

# Run the chain
response = chain.invoke(input=input_)

# Print the response content
print(response.content)

The day after Tuesday is Wednesday.


# Few-Shot Prompt Example

Create a few-shot prompt template with example selector for tasks like antonym generation.

In [69]:

# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=25,  # The maximum length that the formatted examples should be.
)
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

# Format Few-Shot Prompt with Input

Render the few-shot prompt with a specific adjective to see the formatted prompt.

In [70]:
# Test just the prompt formatting
print(dynamic_prompt.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: sunny
Output: gloomy

Input: windy
Output: calm

Input: big
Output:


# Test Few-Shot Prompt with Long Input

Test the few-shot prompt with a long input string to observe truncation or formatting.

In [71]:
long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else"
print(dynamic_prompt.format(adjective=long_string))

Give the antonym of every input

Input: happy
Output: sad

Input: big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else
Output:


# Define a Pydantic Data Structure for Structured Output

Define a Pydantic model for a joke with setup and punchline fields.

In [72]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

# Chain for Structured Output with Pydantic Parsing

Set up a chain that prompts the LLM and parses the output into a structured Pydantic object.

In [73]:
# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
output_parser = JsonOutputParser(pydantic_object=Joke)

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)

chain = prompt | openai_llm | output_parser

chain.invoke({"query": joke_query})

{'setup': "Why couldn't the bicycle find its way home?",
 'punchline': 'Because it lost its bearings!'}

# Comma-Separated List Output Parsing

Use a parser to extract a comma-separated list from the LLM output.

In [74]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="Answer the user query. {format_instructions}\nList five {subject}.",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

chain = prompt | openai_llm | output_parser

# Invoke Chain for List Output

Invoke the chain to get a list of items (e.g., ice cream flavors) from the LLM.

In [75]:
chain.invoke({"subject": "ice cream flavors"})

['Vanilla',
 'Chocolate',
 'Strawberry',
 'Mint Chocolate Chip',
 'Cookies and Cream']

# Create a Document with Metadata

Create a LangChain Document object with page content and metadata fields.

In [76]:
Document(page_content="""Python is an interpreted high-level general-purpose programming language. 
                        Python's design philosophy emphasizes code readability with its notable use of significant indentation.""",
         metadata={
             'my_document_id' : 234234,
             'my_document_source' : "About Python",
             'my_document_create_time' : 1680013019
         })

Document(metadata={'my_document_id': 234234, 'my_document_source': 'About Python', 'my_document_create_time': 1680013019}, page_content="Python is an interpreted high-level general-purpose programming language. \n                        Python's design philosophy emphasizes code readability with its notable use of significant indentation.")

# Create a Document Without Metadata

Create a Document object with only page content.

In [77]:
Document(page_content="""Python is an interpreted high-level general-purpose programming language. 
                        Python's design philosophy emphasizes code readability with its notable use of significant indentation.""")

Document(metadata={}, page_content="Python is an interpreted high-level general-purpose programming language. \n                        Python's design philosophy emphasizes code readability with its notable use of significant indentation.")

# Load a PDF Document

Use PyPDFLoader to load a PDF from a URL and inspect a specific page.

In [78]:
loader = PyPDFLoader("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf")

document = loader.load()

document[2]  # take a look at the page 2

Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2023-12-31T03:50:13+00:00', 'author': 'IEEE', 'moddate': '2023-12-31T03:52:06+00:00', 'title': 's8329 final', 'source': 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf', 'total_pages': 6, 'page': 2, 'page_label': '3'}, page_content='Figure 2. An AIMessage illustration \nC. Prompt Template \nPrompt templates [10] allow you to structure input for LLMs. \nThey provide a convenient way to format user inputs and \nprovide instructions to generate responses. Prompt templates \nhelp ensure that the LLM understands the desired context and \nproduces relevant outputs. \nThe prompt template classes in LangChain are built to \nmake constructing prompts with dynamic inputs easier. Of \nthese classes, the simplest is the PromptTemplate. \nD. Chain \nChains [11] in LangChain refer to the combination of \nmultiple components to achieve specific tasks. Th

# Print Page Content from PDF

Print the first 1000 characters of the content from a specific PDF page.

In [79]:
print(document[1].page_content[:1000])  # print the page 1's first 1000 tokens

LangChain helps us to unlock the ability to harness the 
LLM’s immense potential in tasks such as document analysis, 
chatbot development, code analysis, and countless other 
applications. Whether your desire is to unlock deeper natural 
language understanding , enhance data, or circumvent 
language barriers through translation, LangChain is ready to 
provide the tools and programming support you need to do 
without it that it is not only difficult but also fresh for you. Its 
core functionalities encompass: 
1. Context-Aware Capabilities: LangChain facilitates the 
development of applications that are inherently 
context-aware. This means that these applications can 
connect to a language model and draw from various 
sources of context, such as prompt instructions, a few-
shot examples, or existing content, to ground their 
responses effectively. 
2. Reasoning Abilities: LangChain equips applications 
with the capacity to reason effectively. By relying on a 
language model, these appl

# Load Web Page Content

Use WebBaseLoader to load content from a web page and print a sample.

In [80]:
loader = WebBaseLoader("https://python.langchain.com/v0.2/docs/introduction/")

web_data = loader.load()

print(web_data[0].page_content[:1000])






Introduction | ü¶úÔ∏èüîó LangChain







Skip to main contentA newer LangChain version is out! Check out the latest version.IntegrationsAPI referenceLatestLegacyMorePeopleContributingCookbooks3rd party tutorialsYouTubearXivv0.2Latestv0.2v0.1ü¶úÔ∏èüîóLangSmithLangSmith DocsLangChain HubJS/TS Docsüí¨SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generationBuild a Local RAG ApplicationBuild a PDF ingestion and Question/Answering systemBuild a Retrieval Augmented Generation (RAG) AppVector stores and retrieversBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow to add memory to chatbotsHow to use example selectorsHow to map values to a graph databaseHow to add a semantic layer 

# Split Documents into Chunks

Split loaded documents into smaller chunks for processing and print the number of chunks.

In [81]:
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20, separator="\n")  # define chunk_size which is length of characters, and also separator.
chunks = text_splitter.split_documents(document)
print(len(chunks))

147


# Inspect a Chunk's Content

Print the content of a specific chunk from the split documents.

In [82]:
chunks[5].page_content   # take a look at any chunk's page content

'individuals seeking guidance and support in these critical areas. \nMindGuide lever ages the capabilities of LangChain and its \nChatModels, specifically Chat OpenAI, as the bedrock of its'

# Generate Embeddings for Chunks

Initialize OpenAI embeddings, truncate text, and generate embeddings for document chunks.

In [83]:

# Initialize OpenAI embeddings
openai_embedding = OpenAIEmbeddings(
    model="text-embedding-ada-002", 
    openai_api_key= open_api_key
)

texts = [text.page_content for text in chunks]
# print(f"texts: {texts}")

#`TRUNCATE_INPUT_TOKENS: 3`
truncated_texts = [" ".join(text.split()[:3]) for text in texts]
print(f"truncated_texts: {truncated_texts[0][:5]}")

# Get the embeddings
embedding_result = openai_embedding.embed_documents(truncated_texts)

# Inspect the first embedding vector
print(embedding_result[0][:5])


truncated_texts: * cor
[-0.003708885982632637, -0.026277778670191765, -0.012813132256269455, -0.020346276462078094, 0.01486269012093544]


# Create a Chroma Vector Store

Create a Chroma vector store from the document chunks and embeddings.

In [84]:
docsearch = Chroma.from_documents(chunks, openai_embedding)

# Similarity Search Example

Perform a similarity search in the vector store for a sample query and print the result.

In [85]:
query = "Langchain"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

II. LANGCHAIN 
LangChain, with its open -source essence, emerges as a 
promising solution, aiming to simplify the complex process of 
developing applications powered by large language models


# Create a Retriever from Vector Store

Convert the Chroma vector store into a retriever object.

In [86]:
retriever = docsearch.as_retriever()

# Invoke Retriever for Query

Use the retriever to find relevant documents for a query.

In [87]:
docs = retriever.invoke("Langchain")

# Inspect Retrieved Document

Print the content of the first retrieved document.

In [88]:
docs[0]

Document(metadata={'creationdate': '2023-12-31T03:50:13+00:00', 'page_label': '1', 'total_pages': 6, 'moddate': '2023-12-31T03:52:06+00:00', 'title': 's8329 final', 'author': 'IEEE', 'page': 0, 'source': 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf', 'producer': 'PyPDF', 'creator': 'Microsoft Word'}, page_content='II. LANGCHAIN \nLangChain, with its open -source essence, emerges as a \npromising solution, aiming to simplify the complex process of \ndeveloping applications powered by large language models')

# Parent Document Retrieval Setup

Set up parent and child text splitters, a Chroma vector store, and an in-memory store for parent document retrieval.

In [89]:
# Set two splitters. One is with big chunk size (parent) and one is with small chunk size (child)
parent_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=20, separator='\n')
child_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=20, separator='\n')

vectorstore = Chroma(
    collection_name="split_parents", embedding_function=openai_embedding
)

# The storage layer for the parent documents
store = InMemoryStore()

# Initialize ParentDocumentRetriever

Create a ParentDocumentRetriever using the vector store, docstore, and splitters.

In [90]:
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

# Add Documents to Retriever

Add loaded documents to the parent retriever's docstore.

In [91]:
retriever.add_documents(document)

# Check Number of Stored Parent Documents

Count the number of parent documents stored in the in-memory docstore.

In [92]:
len(list(store.yield_keys()))

16

# Similarity Search on Sub-Documents

Perform a similarity search on the sub-documents in the vector store.

In [93]:
sub_docs = vectorstore.similarity_search("Langchain")

# Print Sub-Document Content

Print the content of a sub-document retrieved from the vector store.

In [94]:
print(sub_docs[0].page_content)

LangChain helps us to unlock the ability to harness the 
LLM’s immense potential in tasks such as document analysis, 
chatbot development, code analysis, and countless other 
applications. Whether your desire is to unlock deeper natural 
language understanding , enhance data, or circumvent 
language barriers through translation, LangChain is ready to


# Retrieve Parent Documents for Query

Use the parent retriever to get relevant parent documents for a query.

In [95]:
retrieved_docs = retriever.invoke("Langchain")

# Print Retrieved Parent Document Content

Print the content of the first parent document retrieved.

In [96]:
print(retrieved_docs[0].page_content)

LangChain helps us to unlock the ability to harness the 
LLM’s immense potential in tasks such as document analysis, 
chatbot development, code analysis, and countless other 
applications. Whether your desire is to unlock deeper natural 
language understanding , enhance data, or circumvent 
language barriers through translation, LangChain is ready to 
provide the tools and programming support you need to do 
without it that it is not only difficult but also fresh for you. Its 
core functionalities encompass: 
1. Context-Aware Capabilities: LangChain facilitates the 
development of applications that are inherently 
context-aware. This means that these applications can 
connect to a language model and draw from various 
sources of context, such as prompt instructions, a few-
shot examples, or existing content, to ground their 
responses effectively. 
2. Reasoning Abilities: LangChain equips applications 
with the capacity to reason effectively. By relying on a 
language model, these appl

# Set Up RetrievalQA Chain

Create a RetrievalQA chain using the LLM and retriever, and invoke it with a question.

In [97]:
from langchain.chains import RetrievalQA

In [98]:
qa = RetrievalQA.from_chain_type(llm=openai_llm, 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=False)
query = "what is this paper discussing?"
qa.invoke(query)

{'query': 'what is this paper discussing?',
 'result': 'This paper is discussing the application of recent interventions for mental disorders, specifically focusing on anxiety, depression, and suicidal thoughts, using techniques that have been effective in text processing tasks.'}

# Import ChatMessageHistory for Conversation Memory

Import the ChatMessageHistory class to manage conversation history.

In [99]:
from langchain.memory import ChatMessageHistory

# Create and Update Conversation History

Create a conversation history object and add AI and user messages.

In [100]:
chat = openai_llm

history = ChatMessageHistory()

history.add_ai_message("hi!")

history.add_user_message("what is the capital of France?")

# Inspect Conversation Messages

Display the list of messages in the conversation history.

In [101]:
history.messages

[AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the capital of France?', additional_kwargs={}, response_metadata={})]

# Invoke LLM with Conversation History

Send the conversation history to the LLM and get a response.

In [102]:
ai_response = chat.invoke(history.messages)
ai_response

AIMessage(content='The capital of France is Paris.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 20, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BOPFPhimFAvckTnHzmjHrELpu7BpI', 'finish_reason': 'stop', 'logprobs': None}, id='run-b1198908-7fc2-463b-874f-b124eb973a58-0', usage_metadata={'input_tokens': 20, 'output_tokens': 8, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# Import Conversation Memory and Chain

Import ConversationBufferMemory and ConversationChain for managing conversational state.

In [103]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Initialize a Conversation Chain

Set up a conversation chain with memory and verbosity enabled.

In [104]:
conversation = ConversationChain(
    llm=openai_llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

# Start a Conversation

Invoke the conversation chain with an initial message.

In [105]:
conversation.invoke(input="Hello, I am a little cat. Who are you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello, I am a little cat. Who are you?
AI:

> Finished chain.


{'input': 'Hello, I am a little cat. Who are you?',
 'history': '',
 'response': 'Hello little cat! I am an artificial intelligence program designed to assist and provide information to users like yourself. I am constantly learning and updating my database to better serve you. How can I help you today?'}

# Continue the Conversation

Send a follow-up message to the conversation chain.

In [106]:
conversation.invoke(input="What can you do?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, I am a little cat. Who are you?
AI: Hello little cat! I am an artificial intelligence program designed to assist and provide information to users like yourself. I am constantly learning and updating my database to better serve you. How can I help you today?
Human: What can you do?
AI:

> Finished chain.


{'input': 'What can you do?',
 'history': 'Human: Hello, I am a little cat. Who are you?\nAI: Hello little cat! I am an artificial intelligence program designed to assist and provide information to users like yourself. I am constantly learning and updating my database to better serve you. How can I help you today?',
 'response': 'I can provide information on a wide range of topics, answer questions, assist with tasks like scheduling appointments or setting reminders, offer recommendations based on your preferences, and even engage in casual conversation like we are doing now. Just let me know what you need help with!'}

# Import SequentialChain for Multi-Step Workflows

Import SequentialChain to build multi-step LLM workflows.

In [107]:
from langchain.chains import LLMChain

# Sequential chain
from langchain.chains import SequentialChain

# Build a Sequential Chain for Movie Recommendation

Create a two-step chain: recommend a classic movie by location, then describe it.

In [108]:
# Prompt 1: Recommend a classic movie
template = """Your job is to come up with a classic movie from the area the user suggests.
Location: {location}

YOUR RESPONSE:"""
prompt_template = PromptTemplate(template=template, input_variables=['location'])

# First chain
location_chain = LLMChain(llm=openai_llm, prompt=prompt_template, output_key='movie')

# Prompt 2: Describe the movie
desc_template = """You are an art critic. Describe the classical "{movie}" in a short paragraph."""
desc_prompt = PromptTemplate(template=desc_template, input_variables=['movie'])

# Second chain
desc_chain = LLMChain(llm=openai_llm, prompt=desc_prompt, output_key='description')

overall_chain = SequentialChain(
    chains=[location_chain, desc_chain],
    input_variables=['location'],
    output_variables=['movie', 'description'],
    verbose=True
)

# Run it
result = overall_chain.invoke({'location': 'Kyoto'})
print("Movie:", result['movie'])
print("Description:", result['description'])




> Entering new SequentialChain chain...

> Finished chain.
Movie: "The Last Samurai" (2003) - This epic historical drama film is set in Kyoto, Japan during the 19th century and follows the story of an American military officer who is captured by samurai warriors and learns the way of the samurai. The film showcases the beautiful landscapes and traditional architecture of Kyoto, making it a classic movie from the area.
Description: "The Last Samurai" is a visually stunning film that captures the essence of traditional Japanese culture and history. The breathtaking landscapes of Kyoto serve as a backdrop for the epic story of a foreign soldier who finds himself immersed in the world of the samurai. The film beautifully portrays the intricate details of Japanese architecture and the rich cultural heritage of the region. With its compelling storyline and captivating visuals, "The Last Samurai" is a classic depiction of honor, loyalty, and the clash of civilizations.


# Import SequentialChain Again (Redundant)

Re-import SequentialChain (already imported above).

In [109]:
from langchain.chains import SequentialChain

# Build a Chain for Classic Dish Recommendation

Create a prompt and chain to recommend a classic dish by location.

In [122]:
template = """Your job is to come up with a classic dish from the area that the users suggests.
                {location}
                
                YOUR RESPONSE:
"""
prompt_template = PromptTemplate(template=template, input_variables=['location'])

# chain 1
location_chain = LLMChain(llm=openai_llm, prompt=prompt_template, output_key='meal')

# Build a Chain for Recipe Generation

Create a prompt and chain to generate a recipe for a given meal.

In [116]:
template = """Given a meal {meal}, give a short and simple recipe on how to make that dish at home.

                YOUR RESPONSE:
"""
prompt_template = PromptTemplate(template=template, input_variables=['meal'])

# chain 2
dish_chain = LLMChain(llm=openai_llm, prompt=prompt_template, output_key='recipe')

# Build a Chain for Cooking Time Estimation

Create a prompt and chain to estimate cooking time for a recipe.

In [117]:
template = """Given the recipe {recipe}, estimate how much time I need to cook it.

                YOUR RESPONSE:
"""
prompt_template = PromptTemplate(template=template, input_variables=['recipe'])

# chain 3
recipe_chain = LLMChain(llm=openai_llm, prompt=prompt_template, output_key='time')

# Build and Run a Multi-Step Chain for Meals

Combine the dish, recipe, and time chains into a sequential workflow and run it.

In [118]:
# overall chain
overall_chain = SequentialChain(chains=[location_chain, dish_chain, recipe_chain],
                                      input_variables=['location'],
                                      output_variables=['meal', 'recipe', 'time'],
                                      verbose= True)

# Import pprint for Pretty Printing

Import the pprint module to display results in a readable format.

In [119]:
from pprint import pprint

# Run the Multi-Step Meal Chain and Print Results

Invoke the meal chain with a location and print the outputs.

In [120]:
pprint(overall_chain.invoke(input={'location':'China'}))



> Entering new SequentialChain chain...

> Finished chain.
{'location': 'China',
 'meal': 'One classic dish from China is Peking duck. This dish originated in '
         'Beijing and is known for its crispy skin and tender meat. The duck '
         'is typically roasted and served with pancakes, scallions, cucumber, '
         'and hoisin sauce. It is a popular dish in Chinese cuisine and is '
         'often served at special occasions and celebrations.',
 'recipe': 'Ingredients:\n'
           '- 1 whole duck\n'
           '- Salt\n'
           '- 1 tablespoon honey\n'
           '- 1 tablespoon soy sauce\n'
           '- 1 tablespoon vinegar\n'
           '- Hoisin sauce\n'
           '- Thin pancakes\n'
           '- Sliced scallions\n'
           '- Sliced cucumber\n'
           '\n'
           'Instructions:\n'
           '1. Preheat the oven to 375°F.\n'
           '2. Clean the duck and pat it dry. Rub salt all over the skin.\n'
           '3. In a small bowl, mix honey, soy s

# Import Summarization Chain

Import the summarization chain from LangChain for document summarization.

In [121]:
from langchain.chains.summarize import load_summarize_chain

# Run Summarization Chain on Web Data

Use the summarization chain to summarize loaded web data.

In [126]:
chain = load_summarize_chain(openai_llm, chain_type="stuff", verbose=False)
response = chain.invoke(web_data)

# Print Summarization Output

Print the output text from the summarization chain.

In [127]:
print(response['output_text'])

LangChain is a framework for developing applications powered by large language models (LLMs). It simplifies the development, productionization, and deployment stages of LLM applications, offering open-source building blocks, components, and third-party integrations. The framework includes libraries such as LangGraph for building stateful agents, LangServe for deploying chains as REST APIs, and LangSmith for debugging and monitoring LLM applications. The documentation provides tutorials, how-to guides, API references, and conceptual guides for developers using the LangChain framework.


# (Empty Cell)
This cell is intentionally left blank.

# Import Python REPL Tool

Import the Python REPL tool for executing Python code within LangChain agents.

In [ ]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

# Initialize Python REPL

Create an instance of the Python REPL for code execution.

In [ ]:
python_repl = PythonREPL()

# Run Python Code in REPL

Execute a simple Python code snippet using the Python REPL.

In [ ]:
python_repl.run("a = 3; b = 1; print(a+b)")

# Import PythonREPLTool

Import the PythonREPLTool for use in LangChain agent toolkits.

In [ ]:
from langchain_experimental.tools import PythonREPLTool

# Create Python REPL Tool List

Create a list of tools including the Python REPL tool for agent use.

In [ ]:
tools = [PythonREPLTool()]

# Import Agent and Tool Classes

Import classes for creating and running LangChain agents with custom tools.

In [ ]:
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain_experimental.utilities import PythonREPL
from langchain.prompts import PromptTemplate


# Create a Custom Python REPL Agent

Set up a custom agent with a Python REPL tool, define instructions, and run an example query.

In [ ]:
# Custom instructions
instructions = """You are an agent designed to write and execute Python code to answer questions.
You have access to a Python REPL, which you can use to execute Python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""


template = """You are an agent designed to write and execute Python code to answer questions.
You have access to a Python REPL, which you can use to execute Python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.

TOOLS:
------

You have access to the following tools:

{tools}

To use a tool, please use the following format:

Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

Thought: Do I need to use a tool? No
Final Answer: [your response here]

Begin!
New input: {input}
{agent_scratchpad}
"""


prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad", "tools", "tool_names"],
    template=template
)

# Tool setup
python_repl = PythonREPL()

tools = [
    Tool(
        name="Python_REPL",
        func=lambda x: python_repl.run(clean_python_input(x)),
        description="A Python shell. Use this to execute Python code."
    )
]


# Create agent
agent = create_react_agent(openai_llm, tools, prompt)

# Create agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

# Run example
response = agent_executor.invoke(input={"input": "What is the 3rd fibonacci number?"})
print(response)